# Imputing and Transforming values

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("titanic.csv")
df.head()

## Inspect the features
Note which are numerical and which are categorical.

### Check for missing values
Which features can be dropped?
Which features may we want to complete/impute?

### Distribution of numerical features
Check for outliers.

In [ ]:
df.describe()

### Distribution of categorical features

In [ ]:
df.describe(include=["O"])

In [ ]:
# check for null (nan) values
df["Embarked"].isnull().value_counts()

### Drop poor features

In [ ]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

### Impute missing values (Embarked)

In [ ]:
df.Embarked.value_counts()

In [ ]:
most_common_port = df.Embarked.value_counts().idxmax()
most_common_port

In [ ]:
df['Embarked'] = df['Embarked'].fillna(most_common_port)
    
df[['Embarked', 'Survived']].groupby(['Embarked']).mean().sort_values(by='Survived', ascending=False)

### One-hot encoding "Embarked"

In [ ]:
df.head()

In [ ]:
# use pandas to one-hot encode "Embarked"

# DEFAULTS:
    # prefix_sep='_' 
    # columns=None   ... will encode all columns with categorical variables
    # drop_first=False
# returns a DataFrame

df = pd.get_dummies(df, columns=["Embarked"])
df.head()

### Convert "Sex" to a binary value

In [ ]:
# Converting a categorical feature to a binary one
df["Sex"] = df["Sex"].map({'male':0, 'female':1})

### Impute missing values (Age)

Note: There is a relationship among Age, Gender, and Pclass. 

Perhaps use the mean Age, across sets of Pclass and Gender combinations, to impute missing Ages.  Alternatively, a random value within 1 standard deviation of the mean Age can be used. The median value can also be used

### Determine mean Age and standard deviation by Gender and Pclass

In [ ]:
# Males (coded as Sex=0) in First Class (coded as Pclass=1)
Age01_mean = df.loc[(df['Sex']==0) & (df['Pclass']==1), 'Age'].mean()
Age01_std = df.loc[(df['Sex']==0) & (df['Pclass']==1), 'Age'].std()
Age01_mean, Age01_std

In [ ]:
# Males in Second Class
Age02_mean = df.loc[(df['Sex']==0) & (df['Pclass']==2), 'Age'].mean()
Age02_std = df.loc[(df['Sex']==0) & (df['Pclass']==2), 'Age'].std()
Age02_mean, Age02_std

In [ ]:
# Males in Third Class
Age03_mean = df.loc[(df['Sex']==0) & (df['Pclass']==3), 'Age'].mean()
Age03_std = df.loc[(df['Sex']==0) & (df['Pclass']==3), 'Age'].std()
Age03_mean, Age03_std

### Review Random Numbers

In [ ]:
np.random.rand(3,3)

In [ ]:
np.random.randn(3,3)

In [ ]:
np.random.randint(10, 15, (3,3))

In [ ]:
np.random.uniform(99,100, (3,3))

### Imputing with a randomly selected age within 1 standard deviation of its group's mean

In [ ]:
# use the mean and std of Males in First Class
# to randomly generate an Age within 1 standard deviation of the mean
Age01_impute = round(np.random.uniform(Age01_mean - Age01_std, Age01_mean + Age01_std))
Age01_impute

In [ ]:
# replace the null values with the imputed age
#df.loc[ (df["Age"].isnull()) & (df.Sex=='male') & (df.Pclass==1),'Age'] = Age01_impute

*Do the same, as above, for 'females'

## Feature Engineering
Perhaps create an "AgeBand" feature by grouping Age within bands (discretization).

In [ ]:
df["Age"].head()

In [ ]:
# Create "AgeBand" feature

#df['AgeBand'] = pd.cut(df['Age'], 5)
#df['AgeBand'] = pd.cut(df['Age'], 4)
#df['AgeBand'] = pd.cut(df['Age'], [0,20,40,60,80])
df['AgeBand'] = pd.cut(df['Age'], [0,20,40,60,80], labels=["child","adult","middle age","elder"])
#df['AgeBand'] = pd.cut(df['Age'], [0,20,40,60,80], labels=[1,2,3,4])
df["AgeBand"].head(20)

In [ ]:
# observe survival rate by age group and gender
df[['AgeBand', 'Survived', "Sex"]].groupby(['AgeBand', "Sex"]).mean().sort_values(by='AgeBand', ascending=True)

Perhaps create a **"FamilySize"** feature, combining "SibSp" and "Parch"

In [ ]:
# Create "FamilySize" feature  and perhaps drop "SibSp" and "Parch"
 
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

df[['FamilySize', 'Survived']].groupby(['FamilySize']).mean().sort_values(by='Survived', ascending=False)

Perhaps create an **"IsAlone"** feature using "FamilySize"

In [ ]:
# Create IsAlone feature
df['IsAlone'] = 0
df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1

df[['IsAlone', 'Survived']].groupby(['IsAlone']).mean()

## Dataset after dropping and engineering features

In [ ]:
df.head()

# Model and predict

**Note:** You must drop or impute any missing values. Scikit-learn will give you an error if you have any NaNs in your data. 

In [ ]:
# ...
